In [1]:
import os
import h5py
import numpy as np
import torch
import torchani
import ase
from ase import Atom
from ase import Atoms
from ase import units
from ase.io import read, write
import matplotlib
from matplotlib import pyplot as plt

device = torch.device('cpu')

home = os.getcwd()

C:\Users\AMJ\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
mol = []
def printenergy(a=mol):
    epot = a.get_potential_energy()
    ekin = a.get_kinetic_energy()
    return epot + ekin

In [3]:
os.chdir(home)

network_folders = []



def atomic(linear=[(384,128), (128,128), (128,64), (64,1)]):
    model = torch.nn.Sequential(
        torch.nn.Linear(*linear[0]),
        torch.nn.CELU(0.1),
        torch.nn.Linear(*linear[1]),
        torch.nn.CELU(0.1),
        torch.nn.Linear(*linear[2]),
        torch.nn.CELU(0.1),
        torch.nn.Linear(*linear[3])
    )
    return model
nn = torchani.ANIModel([atomic(v) for v in (
[(384,160), (160,128), (128,96), (96,1)],
[(384,144), (144,112), (112,96), (96,1)],
[(384,128), (128,112), (112,96), (96,1)],
[(384,128), (128,112), (112,96), (96,1)]
)])

network_folders = ['./best.pt']
with h5py.File('./transfer/energy2.h5', 'r') as f:
    coordinates = f['firstlayer']['secondlayer']['coordinates']
    coordinates = f['firstlayer']['secondlayer']['coordinates'][:len(coordinates)]
    dataset_energies = f['firstlayer']['secondlayer']['energies'][:len(coordinates)]
    
    path = './train/eval/'
    const_file = path + 'rHCNO-5.2R_16-3.5A_a4-8.params'
    sae_file = path + 'sae_linfit_CAS.dat'
 
    consts = torchani.neurochem.Constants(const_file)
    aev_computer = torchani.AEVComputer(**consts)
    energy_shifter = torchani.neurochem.load_sae(sae_file)

In [4]:
print(os.getcwd())
evaluated_energies = []
for i in range(1):
    for x in range(len(coordinates)):
        c = coordinates[x]
        mol = [ Atom('C', (c[0][0],  c[0][1], c[0][2])),
        Atom('C', (c[1][0],  c[1][1], c[1][2])),
        Atom('H', (c[2][0],  c[2][1], c[2][2])),
        Atom('H', (c[3][0],  c[3][1], c[3][2])),
        Atom('H', (c[4][0],  c[4][1], c[4][2])),
        Atom('H', (c[5][0],  c[5][1], c[5][2])) ]
        mol = Atoms(mol)
        personal_directory = os.getcwd()
        network = network_folders
        calculator = torchani.ase.Calculator(personal_directory, consts.species, aev_computer, network, nn, energy_shifter, device)

        mol.set_calculator(calculator)
        energy = float(str(printenergy(a=mol))) * (1/units.Hartree) * 627.509474062112 # Hartree to kcal/mol conversion
        evaluated_energies.append(energy)

dataset_energies = [float(dataset_energies[x]) * 627.509474062112 for x in range(len(dataset_energies))]        

result = []
for i, j in zip(dataset_energies, evaluated_energies):
    result.append(i-j)

c:\Users\AMJ\git_code\goodpast\Workstation_Code\8round_excited_0.4_correct_state


C:\Users\AMJ\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\torchani\ase.py:142: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at ..\torch\csrc\utils\tensor_new.cpp:233.)
  cell = torch.tensor(self.atoms.get_cell(complete=True),


In [5]:

# Plot energies from the dataset and from the network

x_axis = []
y_axis = []
for i in range(len(evaluated_energies)):
    x_axis.append(int(i))
    y_axis.append(round(float(evaluated_energies[i] ), 11))
line1 = plt.plot(x_axis, y_axis)
plt.savefig('./network_energies.png')
plt.clf()
x_axis, y_axis = [], []
for i in range(len(dataset_energies)):
    x_axis.append(int(i))
    y_axis.append(round(float(dataset_energies[i] ), 11))
line1 = plt.plot(x_axis, y_axis)
plt.savefig('./dataset_energies.png')
plt.clf()
x_axis, y_axis = [], []
for i in range(len(evaluated_energies)):
    x_axis.append(int(i))
    y_axis.append(round(float(result[i]), 11))
line1 = plt.plot(x_axis, y_axis)
plt.savefig('./residuals.png')
plt.clf()

<Figure size 640x480 with 0 Axes>